In [1]:
# 기본적인 라이브러리 호출

import sys
import os
import numpy as np
import json

import tensorflow.compat.v1 as tf
from tensorflow.compat.v1 import keras

In [2]:
# 이전에 저장했던 학습에 필요한 디렉터리 설정 및 학습/평가 데이터를 불러온다.

DATA_IN_PATH='./data_in/'
DATA_OUT_PATH='./data_out/'
INPUT_TRAIN_DATA_FILE_NAME='train_input.npy'
LABEL_TRAIN_DATA_FILE_NAME='train_label.npy'

TEST_INPUT_DATA_FILE_NAME='test_input.npy'

DATA_CONFIGS_FILE_NAME='data_configs.json'

train_input_data=np.load(open(DATA_IN_PATH+INPUT_TRAIN_DATA_FILE_NAME,'rb'))
train_label_data=np.load(open(DATA_IN_PATH+LABEL_TRAIN_DATA_FILE_NAME,'rb'))
test_input_data=np.load(open(DATA_IN_PATH+TEST_INPUT_DATA_FILE_NAME,'rb'))

with open(DATA_IN_PATH+DATA_CONFIGS_FILE_NAME,'r') as f:
    prepro_configs=json.load(f)
    print(prepro_configs.keys())

dict_keys(['vocab', 'vocab_size'])


In [3]:
from sklearn.model_selection import train_test_split

# 파라미터 변수

RNG_SEED=1234
BATCH_SIZE=128
NUM_EPOCHS=10000
VOCAB_SIZE=prepro_configs['vocab_size']
EMB_SIZE=128
VALID_SPLIT=0.2

# 학습 데이터와 검증 데이터를 train_test_split 함수를 활용해 나눈다.
train_input, eval_input, train_label, eval_label=train_test_split(train_input_data, train_label_data, test_size=VALID_SPLIT, random_state=RNG_SEED)

In [4]:
# 전처리 학습을 위해 tf.data를 설정한다.

def mapping_fn(X, Y=None):
    input, label={'x':X},Y
    return input,label

def train_input_fn():
    dataset=tf.data.Dataset.from_tensor_slices((train_input, train_label))
    dataset=dataset.shuffle(buffer_size=len(train_input))
    dataset=dataset.batch(BATCH_SIZE)
    dataset=dataset.map(mapping_fn)
    dataset=dataset.repeat(count=NUM_EPOCHS)
    
    iterator=dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def eval_input_fn():
    dataset=tf.data.Dataset.from_tensor_slices((input_eval,label_eval))
    dataset=dataset.shuffle(buffer_size=len(input_eval))
    dataset=dataset.batch(BATCH_SIZE)
    dataset=dataset.map(mapping_fn)
    
    iterator=dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [5]:
def model_fn(features, labels, mode):
    TRAIN=mode==tf.estimator.ModeKeys.TRAIN
    EVAL=mode==tf.estimator.ModeKeys.EVAL
    PREDICT=mode==tf.estimator.ModeKeys.PREDICT
    
    # embedding layer를 선언합니다.
    embedding_layer=keras.layers.Embedding(VOCAB_SIZE,EMB_SIZE)(features['x'])
    
    # embedding layer에 대한 output에 대해 dropout을 취합니다.
    dropout_emb=keras.layers.Dropout(rate=0.5)(embedding_layer)
    
    ## filters=128이고 kernel_size=3,4,5다.
    ## 길이가 3, 4, 5인 128개의 다른 필터를 생성한다. 3,4,5 gram의 효과처럼 다양한 각도에서 문장을 보는 효과가 있다.
    ## conv1d는 (배치 크기, 길이, 채널)로 입력값을 받는데, 배치 사이즈: 문장 숫자 | 길이: 각 문장의 단어의 개수 | 채널 : 임베딩 출력 차원수임
    
    conv1=keras.layers.Conv1D(filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu)(dropout_emb)
    
    pool1=keras.layers.GlobalMaxPool1D()(conv1)
    
    conv2=keras.layers.Conv1D(filters=128, kernel_size=4, padding='valid', activation=tf.nn.relu)(dropout_emb)
    
    pool2=keras.layers.GlobalMaxPool1D()(conv2)
    
    conv3=keras.layers.Conv1D(filters=128, kernel_size=5, padding='valid', activation=tf.nn.relu)(dropout_emb)
    
    pool3=keras.layers.GlobalMaxPool1D()(conv3)
    
    # 3,4,5gram 이후 모아주기
    concat=keras.layers.concatenate([pool1, pool2, pool3])
    
    hidden=keras.layers.Dense(250, activation=tf.nn.relu)(concat)
    dropout_hidden=keras.layers.Dropout(rate=0.5)(hidden)
    logits=keras.layers.Dense(1, name='logits')(dropout_hidden)
    logits=tf.squeeze(logits, axis=-1)
    
    # 최종적으로 학습, 검증, 평가의 단계로 나누어 활용
    
    if PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions={'prob':tf.nn.sigmoid(logits)})
    
    loss=tf.losses.sigmoid_cross_entropy(labels, logits)
    
    if EVAL:
        pred=tf.nn.sigmoid(logits)
        accuracy=tf.metrics.accuracy(labels, tf.round(pred))
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc':accuracy})
    
    if TRAIN:
        global_step=tf.train.get_global_step()
        train_op=tf.train.AdamOptimizer(0.001).minimize(loss, global_step)
        
        return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss=loss)

In [ ]:
model_dir=os.path.join(os.getcwd(),"data_out/checkpoint/cnn/")
os.makedirs(model_dir, exist_ok=True)

# Estimator 객체 생성
cnn_est=tf.estimator.Estimator(model_fn, model_dir=model_dir)
# 학습하기
cnn_est.train(train_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\mywork\\NLP\\data_out/checkpoint/cnn/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automat

INFO:tensorflow:loss = 3.5924172e-06, step = 4800 (27.144 sec)
INFO:tensorflow:global_step/sec: 3.95905
INFO:tensorflow:loss = 3.493723e-06, step = 4900 (25.259 sec)
INFO:tensorflow:global_step/sec: 3.98683
INFO:tensorflow:loss = 3.9908728e-06, step = 5000 (25.091 sec)
INFO:tensorflow:global_step/sec: 3.9528
INFO:tensorflow:loss = 3.2160276e-06, step = 5100 (25.290 sec)
INFO:tensorflow:global_step/sec: 3.65866
INFO:tensorflow:loss = 3.2543455e-06, step = 5200 (27.333 sec)
INFO:tensorflow:global_step/sec: 3.91883
INFO:tensorflow:loss = 2.6232224e-06, step = 5300 (25.517 sec)
INFO:tensorflow:global_step/sec: 4.01138
INFO:tensorflow:loss = 2.4389801e-06, step = 5400 (24.930 sec)
INFO:tensorflow:global_step/sec: 4.0121
INFO:tensorflow:loss = 2.5648562e-06, step = 5500 (24.932 sec)
INFO:tensorflow:global_step/sec: 3.50942
INFO:tensorflow:loss = 1.8401024e-06, step = 5600 (28.488 sec)
INFO:tensorflow:global_step/sec: 3.73642
INFO:tensorflow:loss = 1.8400646e-06, step = 5700 (26.764 sec)
INFO

INFO:tensorflow:global_step/sec: 3.56344
INFO:tensorflow:loss = 6.159111e-08, step = 11700 (28.062 sec)
INFO:tensorflow:global_step/sec: 3.89008
INFO:tensorflow:loss = 7.37486e-08, step = 11800 (25.707 sec)
INFO:tensorflow:global_step/sec: 3.98119
INFO:tensorflow:loss = 6.6259375e-08, step = 11900 (25.118 sec)
INFO:tensorflow:global_step/sec: 4.00186
INFO:tensorflow:loss = 3.6151775e-08, step = 12000 (24.988 sec)
INFO:tensorflow:global_step/sec: 3.73222
INFO:tensorflow:loss = 5.6634107e-08, step = 12100 (26.794 sec)
INFO:tensorflow:global_step/sec: 3.57525
INFO:tensorflow:loss = 5.4141474e-08, step = 12200 (27.969 sec)
INFO:tensorflow:global_step/sec: 3.77562
INFO:tensorflow:loss = 3.899367e-08, step = 12300 (26.486 sec)
INFO:tensorflow:global_step/sec: 3.86838
INFO:tensorflow:loss = 4.851951e-08, step = 12400 (25.851 sec)
INFO:tensorflow:global_step/sec: 3.93735
INFO:tensorflow:loss = 2.9146939e-08, step = 12500 (25.399 sec)


In [ ]:
# 평가하기
enn_est.evaluate(eval_input_fn)

In [ ]:
DATA_IN_PATH='./data_in/'
DATA_OUT_PATH='./data_out/'
TEST_INPUT_DATA='test_input.npy'
TEST_ID_DATA='test_id.npy'

test_input_data=np.load(open(DATA_IN_PATH+TEST_INPUT_DATA,'rb'))
ids=np.load(open(DATA_IN_PATH+TEST_ID_DATA,'rb'))

print(eval_data)